<a href="https://colab.research.google.com/github/mnijhuis-dnb/open_source_workshop/blob/master/OSW_Pandas_us_hours_worked_verwerkt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import requests

def download_files(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
file_NAICS = '/NAICS.json'
file_Global_mobility_report = '/Global_mobility_report.csv'
file_US_Hours_Worked = '/US_Hours_Worked.json'
file_PPP_Loan_data = '/PPP_Loan_data.csv'
file_US_zips = '/uszips.xslx'

download_files('1Fe73x8Splw9xY5FOTnpLWENJaBU2tHEy', file_NAICS)
download_files('1d4C6sTuD53tc_Ewbfb1iXascKzDSzDl0', file_Global_mobility_report)
download_files('1KzT1XNA9K0enUv73nwPSkBYp7btg0pvJ', file_US_Hours_Worked)
download_files('1_7s1tH7FVNA5s5gu1vcWxikmGYQ2ryvn', file_PPP_Loan_data)
download_files('1f-ru7e4Ml0d5uzOMGtNpmOQ3kGSr7iT_', file_US_zips)

In [ ]:
df_Hours_Worked = pd.read_json(file_US_Hours_Worked)
df_Hours_Worked = pd.merge(df_Hours_Worked, 
              pd.DataFrame(
                  [list(row.values()) for row in df_Hours_Worked['Results']]
                  ,columns=list(df_Hours_Worked['Results'][1]))
              ,left_index=True,right_index=True).drop(columns=['Results'])
              
df_Hours_Worked = pd.merge(df_Hours_Worked, 
              pd.DataFrame(
                  [[dicts['value'] for dicts in row] for row in df_Hours_Worked['data']]
                 ,columns=[dicts['year'] + dicts['period'] for dicts in df_Hours_Worked['data'][77]])
              ,left_index=True,right_index=True).drop(columns=['data'])
df_Hours_Worked['NAICS'] = pd.to_numeric(df_Hours_Worked['NAICS'], errors='coerce')